This code can track the movement of elements in real time using the live feed of the computer's camera

In [1]:
#Perform base imports
import numpy as np
import cv2

In [2]:
#determine strongest corners in the image
corner_track_parameters=dict(maxCorners=20, #how many corners to detect
                             qualityLevel=0.4, #minimum quality of each corner
                             minDistance=7, #minimum distance between each corner to be returned (Euclidean)
                             blockSize=7 #size of average block for checking covariance among pixel region
                            )

In [3]:
lucas_parameters=dict(winSize=(200,200), #default size
                      maxLevel=2, #find movement at various resolutions
                      criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 15, 0.03))
#criteria count=max number of iterations (max number of search for points) of 15
#epsilon=speed/when to finish search of 0.03

In [9]:
#facial tracking on camera

cap=cv2.VideoCapture(0) #gets live video from computer camera

ret, previous_frame=cap.read()

previous_gray=cv2.cvtColor(previous_frame,cv2.COLOR_RGB2GRAY) #converts the frame to grayscale

##CHOOSING POINTS TO TRACK##
previous_points=cv2.goodFeaturesToTrack(previous_gray, mask=None,**corner_track_parameters)
#pass in the image and the previously established dictionary of parameters to cv2's feature tracking mechanism
mask=np.zeros_like(previous_frame) #create array of zeros in the same shape as the frame in order to draw on it and display points on it

while True:
    ret, frame=cap.read()
    frame_grayscale=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    next_points, status, error=cv2.calcOpticalFlowPyrLK(previous_gray,frame_grayscale,previous_points, None, **lucas_parameters)
    #calculates optical flow (movement of points) using lucas-kanade method
    good_newpoints=next_points[status==1] #if status is 1, movement was detected
    good_previouspoints=previous_points[status==1]
    
    for i, (new, previous) in enumerate(zip(good_newpoints, good_previouspoints)):
        #calculates each x and y position of the points found in order to draw markers on the image
        x_newpoint, y_newpoint=new.ravel() #flattens the array
        x_previouspoint, y_previouspoint=previous.ravel()
        
        ##DRAW LINES ON THE MASK OF ZEROS WE CREATED BEFORE##
        mask=cv2.line(mask, #what image to draw the line on
                      (int(x_newpoint), int(y_newpoint)), (int(x_previouspoint), int(y_previouspoint)), #where to draw the line from and to
                      (0,0,255), #line color
                      4) #line thickness
                      #the above code draws a line that tracks the movement of the old point to the current point
        #draw dot on current frame on the point we're tracking
        frame=cv2.circle(frame, #what image to draw the point on
                         (int(x_newpoint), int(y_newpoint)), #where to draw the center of the circle (this is essentially where the circle will be on the image)
                         8, #radius of the circle
                         (0,0,255), #color of the circle
                         -1) #thickness of the circle (-1 fills in the circle)
    img=cv2.add(frame, mask) #adds the mask to the image
    cv2.imshow('tracking', img)
    
    k=cv2.waitKey(30) & 0xFF
    if k==27:
        break #allows us to use the escape key to escape from the camera
        
    
    #update previous points to be the current points
    previous_gray=frame_grayscale.copy()
    previous_points=good_newpoints.reshape(-1,1,2)

    
##END THE PROGRAM##
cv2.destroyAllWindows()
cap.release()
    
    